In [1]:
from utils import *
from massfunction import *

import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import os
import emcee
import sys
import numpy as np
import pickle

In [2]:
cosmos_f = open('data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

In [3]:
X = []
Y = []
idx = {}
cidx = 0
for box in cosmo_params:
    X += [box]
    idx[box] = cidx
    cidx+=1
    Y += [[cosmo_params[box][key] for key in cosmo_params[box]]]
    
X = np.array(X)
Y = np.array(Y)

from sklearn.preprocessing import StandardScaler

# Assuming Y is your input matrix with shape (n_samples, n_features)
# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(Y)

# Standardize the data
Y = scaler.transform(Y)

In [11]:
def find_most_central_datapoint_index(Y):
    centroid = np.mean(Y, axis=0)
    distances = np.linalg.norm(Y - centroid, axis=1)
    most_central_index = np.argmin(distances)
    return most_central_index
X[find_most_central_datapoint_index(Y)]

'Box_n50_0_1400'

In [4]:
#TODO remove boxes thta crash when prepare data

In [5]:
def dist(a,b):
    return np.sqrt(np.sum(np.square(Y[b]-Y[a])))

edges = [[dist(i,j) for i in range(len(X))] for j in range(len(X))]

In [6]:
import sys

def find_min_key(key, mst_set, n):
    min_key = sys.maxsize
    min_index = -1
    for i in range(n):
        if key[i] < min_key and not mst_set[i]:
            min_key = key[i]
            min_index = i
    return min_index

def prim_mst(edges, n):
    parent = [None] * n
    key = [sys.maxsize] * n
    mst_set = [False] * n

    key[0] = 0  # Start with the first node as the root
    parent[0] = -1  # Root has no parent

    for _ in range(n - 1):
        u = find_min_key(key, mst_set, n)
        mst_set[u] = True

        for v in range(n):
            if edges[u][v] > 0 and not mst_set[v] and edges[u][v] < key[v]:
                key[v] = edges[u][v]
                parent[v] = u

    mst = [[0] * n for _ in range(n)]
    for i in range(1, n):
        mst[i][parent[i]] = edges[i][parent[i]]
        mst[parent[i]][i] = edges[parent[i]][i]

    return mst

n = len(edges)

mst = prim_mst(edges, n)

In [7]:
from collections import deque

def bfs_traversal(mst, start):
    tot = 0
    n = len(mst)
    visited = [False] * n
    parent = [None] * n

    queue = deque()
    queue.append(start)
    visited[start] = True

    while queue:
        node = queue.popleft()
        print("%-10s"%(X[node]), end='\t ')
        if parent[node] is not None:
            prev = parent[node]
            dist = np.sqrt(np.sum(np.square(np.array(Y[idx[X[node]]])- np.array(Y[idx[X[prev]]]))))
            tot += dist

            print("Prev: %-10s\t distance:%.2f"%(X[parent[node]], dist),  end='')
        print()

        for neighbor in range(n):
            if mst[node][neighbor] > 0 and not visited[neighbor]:
                queue.append(neighbor)
                visited[neighbor] = True
                parent[neighbor] = node
    assert(all(visited))
    print(tot)

start_node = idx['Box_n50_0_1400'] # Starting node for traversal

bfs_traversal(mst, start_node)


Box_n50_0_1400	 
Box_n50_4_1400	 Prev: Box_n50_0_1400	 distance:1.23
Box_n50_14_1400	 Prev: Box_n50_0_1400	 distance:1.06
Box_n50_21_1400	 Prev: Box_n50_0_1400	 distance:1.19
Box_n50_33_1400	 Prev: Box_n50_0_1400	 distance:1.07
Box_n50_35_1400	 Prev: Box_n50_0_1400	 distance:1.07
Box_n50_43_1400	 Prev: Box_n50_0_1400	 distance:1.27
Box_n50_17_1400	 Prev: Box_n50_4_1400	 distance:1.28
Box_n50_34_1400	 Prev: Box_n50_4_1400	 distance:1.12
Box3_1400 	 Prev: Box_n50_14_1400	 distance:1.53
Box_n50_36_1400	 Prev: Box_n50_21_1400	 distance:0.87
Box_n50_19_1400	 Prev: Box_n50_33_1400	 distance:1.22
Box_n50_37_1400	 Prev: Box_n50_33_1400	 distance:0.80
Box_n50_46_1400	 Prev: Box_n50_33_1400	 distance:1.31
Box_n50_1_1400	 Prev: Box_n50_35_1400	 distance:0.98
Box_n50_26_1400	 Prev: Box_n50_35_1400	 distance:1.17
Box_n50_6_1400	 Prev: Box_n50_43_1400	 distance:1.09
Box_n50_32_1400	 Prev: Box_n50_43_1400	 distance:1.09
Box_n50_44_1400	 Prev: Box_n50_43_1400	 distance:0.92
Box97_1400	 Prev: Box_n50_1